In [1]:
import json 
import tensorflow as tf
import numpy as np
import os
import pandas as pd

SEED = 1234
tf.random.set_seed(SEED)

# Get current working directory
cwd = os.getcwd()


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#Data
from google.colab import drive
drive.mount('/content/drive') 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

In [5]:
#datagenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator


apply_data_augmentation = False

# Create training ImageDataGenerator object

if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        validation_split=0.2,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.2)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)




dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir= os.path.join(dataset_dir,"training")

#Batch size
BS = 8

#Image shape
IMG_H = 407 #612
IMG_W = 407

num_classes=3


with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic=json.load(f)
  dataframe= pd.DataFrame(dic.items())
  dataframe.rename(columns={0:'filename', 1:'class'}, inplace=True )
  dataframe=dataframe.sample(frac=1, random_state=SEED)
  dataframe["class"]=dataframe["class"].astype('string')
  train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                subset='training',
                                                image_size=(407,407))
  
  validation_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                 subset='validation',
                                                image_size=(407,407))
  




Found 4492 validated image filenames belonging to 3 classes.
Found 1122 validated image filenames belonging to 3 classes.


In [6]:
#create Dataset Object

#Training


train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))


train_dataset = train_dataset.repeat()

#Validation

valid_dataset = tf.data.Dataset.from_generator(lambda: validation_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()


In [7]:
#VGG model
vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(IMG_H, IMG_W, 3))
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 407, 407, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 407, 407, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 407, 407, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 203, 203, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 203, 203, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 203, 203, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 101, 101, 128)     0     

In [8]:
# Architecture: VGG -> Classifier

#Finetuning
finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False

#Create model
model = tf.keras.Sequential() #we implement this in a sequential way 
model.add(vgg)

# Classifier
#I vectorize the volume and apply classifier 
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

In [10]:
# Optimization params


# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics


metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [11]:
import os
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'First'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [12]:
model.fit(x=train_dataset,
          epochs=30,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(validation_gen),
          callbacks=callbacks)

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

Epoch 1/30
71/71 [==============================] - 138s 2s/step - loss: 0.9625 - accuracy: 0.5165 - val_loss: 0.6903 - val_accuracy: 0.6845
Epoch 2/30
71/71 [==============================] - 131s 2s/step - loss: 0.5111 - accuracy: 0.7863 - val_loss: 0.5882 - val_accuracy: 0.7317
Epoch 3/30
71/71 [==============================] - 131s 2s/step - loss: 0.3291 - accuracy: 0.8791 - val_loss: 0.5383 - val_accuracy: 0.7576
Epoch 4/30
71/71 [==============================] - 131s 2s/step - loss: 0.1996 - accuracy: 0.9401 - val_loss: 0.5415 - val_accuracy: 0.7629
Epoch 5/30
71/71 [==============================] - 132s 2s/step - loss: 0.1135 - accuracy: 0.9784 - val_loss: 0.5389 - val_accuracy: 0.7834
Epoch 6/30
71/71 [==============================] - 131s 2s/step - loss: 0.0677 - accuracy: 0.9904 - val_loss: 0.5620 - val_accuracy: 0.7701
Epoch 7/30
71/71 [==============================] - 131s 2s/step - loss: 0.0426 - accuracy: 0.9955 - val_loss: 0.5878 - val_accuracy: 0.7709
Epoch 8/30
71

In [13]:
import os
from datetime import datetime

from os import listdir


def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1





Found 450 validated image filenames belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
8/8 [==============================] - 7s 878ms/step


In [14]:
create_csv(results, '/content/drive/My Drive')